In [1]:
import kagglehub
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim

In [2]:
path=kagglehub.dataset_download("zalando-research/fashionmnist")

100%|██████████| 68.8M/68.8M [00:00<00:00, 241MB/s]


Extracting files...


In [3]:
import os
os.listdir(path)

['fashion-mnist_test.csv',
 'train-images-idx3-ubyte',
 't10k-labels-idx1-ubyte',
 'train-labels-idx1-ubyte',
 't10k-images-idx3-ubyte',
 'fashion-mnist_train.csv']

In [4]:
train_df=pd.read_csv(path + "/fashion-mnist_train.csv")
test_df=pd.read_csv(path + "/fashion-mnist_test.csv")

In [5]:
device= "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
device

'cuda'

In [7]:
x_train=torch.tensor(train_df.drop("label",axis=1).values,dtype=torch.float32)
y_train=torch.tensor(train_df["label"].values,dtype=torch.long)
x_test=torch.tensor(test_df.drop("label",axis=1).values,dtype=torch.float32)
y_test=torch.tensor(test_df["label"].values,dtype=torch.long)

In [8]:
y_train

tensor([2, 9, 6,  ..., 8, 8, 7])

In [9]:
class CustomDataset(Dataset):
    def __init__(self,x,y):
        self.x=x.reshape(-1,1,28,28)
        self.y=y
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self,index):
        return self.x[index],self.y[index]
train_dataset=CustomDataset(x_train,y_train)
test_dataset=CustomDataset(x_test,y_test)

In [10]:
train_load=DataLoader(train_dataset,batch_size=128,shuffle=True)
test_load=DataLoader(test_dataset,batch_size=128,shuffle=False)

In [25]:
class MYNN(nn.Module):
    def __init__(self,num):
        super().__init__()
        self.convo=nn.Sequential(
            nn.Conv2d(num,32,kernel_size=3,padding="same"),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2,stride=2),

            nn.Conv2d(32,64,kernel_size=3,padding="same"),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.classification=nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(64,10)

        )
    def forward(self,x):
        x=self.convo(x)
        x=self.classification(x)
        return x

In [26]:
learning_rate=0.001
epochs=50
model=MYNN(1)
model.to(device)
optimizer=optim.Adam(model.parameters(),lr=learning_rate,weight_decay=1e-4)
criterion=nn.CrossEntropyLoss()

In [27]:
for epoch in range(epochs):
    total_loss=0
    for x,y in train_load:
        x,y=x.to(device),y.to(device)
        out=model(x)

        loss=criterion(out,y)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        total_loss+=loss.item()
    print(total_loss/len(train_load))


0.4894238363451032
0.3049798350153701
0.25726965182562117
0.2273940413014721
0.20843027274745868
0.19218487294116762
0.1765681189864175
0.16904852459076117
0.15444977553700334
0.14661813889548722
0.13986326703654806
0.13182003574489531
0.12578825150598597
0.11766937095473316
0.11741029431443734
0.11341160720885435
0.1074853734707019
0.1024656464129305
0.09907594541989283
0.10110842071568915
0.09224406079347454
0.09105461305861216
0.08634990018639546
0.08741464675553064
0.0875151415041753
0.08639885659323636
0.08304841111876818
0.0814421836739537
0.07811617053377984
0.07785455828337971
0.0751487985312907
0.07830458314918569
0.07775546124439313
0.07289508819135267
0.07209500183623388
0.07429504284718588
0.07029283212037134
0.07000845400040656
0.06875996049970134
0.06874527675367749
0.06808725861844414
0.06750455219497177
0.06980959774016826
0.06537192349117607
0.06773952206473614
0.06625151493922193
0.06439517754644378
0.06319412697495809
0.061878556389607854
0.06263762601275942


In [28]:
model.eval()

MYNN(
  (convo): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classification): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [29]:
total=0
correct=0
with torch.no_grad():
    for x,y in test_load:
        x,y=x.to(device),y.to(device)
        out=model(x)
        _,predicted=torch.max(out,1)
        total+=x.size(0)
        correct+=(predicted==y).sum().item()
    print(correct/total)

0.9277
